In [2]:
# ✅ Set Gemini API key securely (Google Colab specific)
import os

from dotenv import load_dotenv

load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

from app.prompts import LEARNING_PLANNER
from app.schema import LearningPlan

In [3]:

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.5, max_retries=2, api_key=gemini_api_key)
llm = llm.with_structured_output(LearningPlan)

system_message = LEARNING_PLANNER.format(plan=None)

response = llm.invoke([("system", system_message), ("user", "how to be a pro drummer")])

print(response)

topic='pro drummer' duration_weeks=4 weekly_plans=[WeekPlan(week_number=1, focus='Developing fundamental drumming skills and musicality', activities=[Activity(description='Daily practice of rudiments (single stroke roll, double stroke roll, paradiddle, etc.)', frequency='Daily'), Activity(description='Listen to and transcribe drum parts from various genres (rock, jazz, funk, Latin)', frequency='5x/week'), Activity(description='Work on sight-reading exercises', frequency='3x/week')], resources=[Resource(name='Stick Control for the Snare Drummer by George Lawrence Stone', type='book', url=None), Resource(name='Modern Drummer magazine', type='other', url='https://www.moderndrummer.com/')], checkpoint='Record a video of yourself playing a basic drum beat with good time and feel.'), WeekPlan(week_number=2, focus='Improving groove, fills, and independence', activities=[Activity(description='Practice playing along with backing tracks in different styles', frequency='Daily'), Activity(descript

In [4]:
# ✅ 🔧 MISSING FUNCTION: generate_learning_path()
def generate_learning_path(topic):
    response = llm.invoke([("system", system_message), ("user", topic)])
    return response.model_dump_json(indent=2)


# ✅ Optional test to see if Gemini works (can comment out later)
print(generate_learning_path("Python"))

{
  "topic": "Python",
  "duration_weeks": 4,
  "weekly_plans": [
    {
      "week_number": 1,
      "focus": "Understanding basic Python syntax and data types",
      "activities": [
        {
          "description": "Read introductory chapter on Python syntax",
          "frequency": "Once"
        },
        {
          "description": "Complete basic Python tutorial on data types",
          "frequency": "Daily"
        },
        {
          "description": "Practice writing simple programs using variables and operators",
          "frequency": "3 times a week"
        }
      ],
      "resources": [
        {
          "name": "Python Crash Course",
          "type": "book",
          "url": null
        },
        {
          "name": "Codecademy Python Course",
          "type": "app",
          "url": "https://www.codecademy.com/learn/learn-python-3"
        }
      ],
      "checkpoint": "Submit a program that calculates the area of a rectangle"
    },
    {
      "week_number

In [ ]:
# ✅ Gradio interface
import gradio as gr


def chatbot(topic):
    if not topic.strip():
        return "Please enter a topic."
    return generate_learning_path(topic)


def download_learning_path(topic):
    plan = generate_learning_path(topic)
    filename = f"{topic}_learning_path.txt"
    with open(filename, "w") as f:
        f.write(plan)
    return filename


with gr.Blocks() as demo:
    gr.Markdown("## 📘 Gemini AI Learning Path Generator")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic")
        generate_btn = gr.Button("Generate Learning Path")

    output = gr.Textbox(label="Learning Path", lines=20)
    download_btn = gr.Button("Download")

    generate_btn.click(chatbot, inputs=topic_input, outputs=output)
    download_btn.click(download_learning_path, inputs=topic_input, outputs=None)

demo.launch(share=True, debug=True)